In [18]:
import argparse
import os
import ruamel.yaml as yaml
import numpy as np
import random
import time
import datetime
import json
from pathlib import Path
import json
import pickle
import pandas as pd

import torch
torch.cuda.empty_cache()
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
import torch.distributed as dist
from tqdm import tqdm


from models.blip_nlvr import blip_nlvr
from transformers import  AutoTokenizer
import utils
from utils import cosine_lr_schedule, warmup_lr_schedule
from data import create_dataset, create_sampler, create_loader

In [19]:
config = yaml.load(open("./configs/nlvr.yaml", 'r'), Loader=yaml.Loader)
datasets = create_dataset('nlvr', config) 




Using downloaded and verified file: /mnt/localdata/karoui/datasets/nlvr2/annotations/nlvr_train.json
Using downloaded and verified file: /mnt/localdata/karoui/datasets/nlvr2/annotations/nlvr_dev.json
Using downloaded and verified file: /mnt/localdata/karoui/datasets/nlvr2/annotations/nlvr_test.json


In [20]:
samplers = [None, None, None]
    
batch_size=[config['batch_size_train'],config['batch_size_test'],config['batch_size_test']]
_, val_loader,_ = create_loader(datasets,samplers,batch_size=[64,2, 2],num_workers=[4,4,4],is_trains=[False,False,False], collate_fns=[None,None,None])

#### Model #### 
print("Creating model")
model = blip_nlvr(pretrained=config['pretrained'], image_size=config['image_size'], 
                     vit=config['vit'], vit_grad_ckpt=config['vit_grad_ckpt'], vit_ckpt_layer=config['vit_ckpt_layer'])

Creating model
load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_nlvr.pth
missing keys:
[]


In [25]:
device = 'cuda'
model = model.to(device)

In [41]:
for _, _, text, _ in tqdm(val_loader):
        
    text = model.tokenizer(text, padding='longest', return_tensors="pt").to(device) 
    text.input_ids[:,0] = model.tokenizer.enc_token_id 
    att = text.attention_mask
    output = model.text_encoder(text.input_ids, 
                                    attention_mask = text.attention_mask,   
                                    return_dict = True,
                                    mode="text"
                                )  
    embs = output.last_hidden_state
    sampleLength = att.sum(dim=-1, keepdims=True) 
    maskedEmbs = embs * torch.unsqueeze(att, -1) 
    result =  maskedEmbs.sum(dim=1) / sampleLength
    features_np = result.cpu().data.numpy()
    features_df = pd.DataFrame(features_np) 
    features_df.to_csv('features_test.csv', mode='a', header=False)

    #features_df.to_csv('features_dev.csv', mode='a', header=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 110/110 [02:46<00:00,  1.52s/it]


In [35]:
embs.shape

torch.Size([64, 41, 768])

In [38]:
maskedEmbs.shape

torch.Size([64, 41, 768])

In [40]:
result.shape

torch.Size([64, 768])

In [43]:
len(datasets[1])

6982

In [ ]:
import pandas as